In [1]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import pickle
import typing

Load Data



In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
notebook_directory = os.getcwd()


Loading data from 1994 to 2010

I used standardized data of death rate for a better interpretation and comparison between countries with very different population numbers

In [3]:
# Create the path for the dataset
# data_path = os.path.join('/content', 'drive', 'MyDrive', 'dataset') #, 'DOPP'
data_path = os.path.join(notebook_directory, 'dataset')
death_data1_path = os.path.join(data_path, 'causesofdeath19942010.csv')

# Load the data into a pandas DataFrame
df = pd.read_csv(death_data1_path)

# Display the first few rows of the DataFrame
df.head()

,DATAFLOW,LAST UPDATE,freq,unit,sex,age,icd10,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:HLTH_CD_ASDR(1.0),17/04/20 23:00:00,A,RT,T,TOTAL,A-R_V-Y,AL,2004,1511.8,NaN
1,ESTAT:HLTH_CD_ASDR(1.0),17/04/20 23:00:00,A,RT,T,TOTAL,A-R_V-Y,AT,1994,1360.6,NaN
2,ESTAT:HLTH_CD_ASDR(1.0),17/04/20 23:00:00,A,RT,T,TOTAL,A-R_V-Y,AT,1995,1348.8,NaN
3,ESTAT:HLTH_CD_ASDR(1.0),17/04/20 23:00:00,A,RT,T,TOTAL,A-R_V-Y,AT,1996,1331.4,NaN
4,ESTAT:HLTH_CD_ASDR(1.0),17/04/20 23:00:00,A,RT,T,TOTAL,A-R_V-Y,AT,1997,1300.5,NaN


In [4]:
# Columns to delete
columns_to_delete = ["DATAFLOW", "LAST UPDATE", "freq", "unit", "sex", "age", "OBS_FLAG"]

# Remove the specified columns
df = df.drop(columns_to_delete, axis=1)

In [5]:
df.head()

,icd10,geo,TIME_PERIOD,OBS_VALUE
0,A-R_V-Y,AL,2004,1511.8
1,A-R_V-Y,AT,1994,1360.6
2,A-R_V-Y,AT,1995,1348.8
3,A-R_V-Y,AT,1996,1331.4
4,A-R_V-Y,AT,1997,1300.5


In [6]:
# Create a MultiIndex for rows with 'TIME_PERIOD' and 'geo'
rows_multiIndex = df.set_index(['TIME_PERIOD', 'geo'])

# Use MultiIndex to organize the data in the required format
df_final1 = rows_multiIndex.pivot_table(index=['TIME_PERIOD', 'geo'], columns=['icd10'], values='OBS_VALUE', aggfunc='mean', fill_value=0, dropna=False)

# Display the result
df_final1.head()

icd10            A-R_V-Y   ACC   A_B  C00-D48  D50-D89     E     F  F10   G_H  \
TIME_PERIOD geo                                                                 
1994        AL       0.0   0.0   0.0      0.0      0.0   0.0   0.0  0.0   0.0   
            AT    1360.6  46.6   5.6    319.3      2.0  28.5   3.8  2.5  20.1   
            BE    1334.0  49.3  16.7    346.8      6.3  29.1  28.5  2.9  38.4   
            BG       0.0   0.0   0.0      0.0      0.0   0.0   0.0  0.0   0.0   
            CH    1146.3  52.7  17.0    306.8      3.9  30.0   6.0  2.0  23.1   

icd10                I      J     K    L    M     N  TOXICO  V_Y85  \
TIME_PERIOD geo                                                      
1994        AL     0.0    0.0   0.0  0.0  0.0   0.0     0.0    0.0   
            AT   748.1   60.8  64.2  0.7  2.4  14.1     0.7   17.0   
            BE   517.5  123.8  55.2  6.7  8.1  21.0     0.6   18.8   
            BG     0.0    0.0   0.0  0.0  0.0   0.0     0.0    0.0   
            CH   510.4   85.9  34.8  0.5  6.8  11.8     2.0   11.1   

icd10            X60-X84_Y870  X85-Y09_Y871  
TIME_PERIOD geo                              
1994        AL            0.0           0.0  
            AT           24.4           1.2  
            BE           22.5           1.8  
            BG            0.0           0.0  
            CH           23.0           1.3

In [7]:
# Create a MultiIndex for columns with all unique codes for the main causes of death
columns_icd10 = df['icd10'].unique()
columns_multiIndex = pd.MultiIndex.from_product([columns_icd10], names=['causes of death'])

# Assign the MultiIndex to the DataFrame columns
df_final1.columns = columns_multiIndex

# Display the result
df_final1.head()

causes of death A-R_V-Y   ACC   A_B C00-D48 D50-D89     E     F  F10   G_H  \
TIME_PERIOD geo                                                              
1994        AL      0.0   0.0   0.0     0.0     0.0   0.0   0.0  0.0   0.0   
            AT   1360.6  46.6   5.6   319.3     2.0  28.5   3.8  2.5  20.1   
            BE   1334.0  49.3  16.7   346.8     6.3  29.1  28.5  2.9  38.4   
            BG      0.0   0.0   0.0     0.0     0.0   0.0   0.0  0.0   0.0   
            CH   1146.3  52.7  17.0   306.8     3.9  30.0   6.0  2.0  23.1   

causes of death      I      J     K    L    M     N TOXICO V_Y85 X60-X84_Y870  \
TIME_PERIOD geo                                                                 
1994        AL     0.0    0.0   0.0  0.0  0.0   0.0    0.0   0.0          0.0   
            AT   748.1   60.8  64.2  0.7  2.4  14.1    0.7  17.0         24.4   
            BE   517.5  123.8  55.2  6.7  8.1  21.0    0.6  18.8         22.5   
            BG     0.0    0.0   0.0  0.0  0.0   0.0    0.0   0.0          0.0   
            CH   510.4   85.9  34.8  0.5  6.8  11.8    2.0  11.1         23.0   

causes of death X85-Y09_Y871  
TIME_PERIOD geo               
1994        AL           0.0  
            AT           1.2  
            BE           1.8  
            BG           0.0  
            CH           1.3

Loading data from 2011 to 2022

In [8]:
# Create the path for the dataset
death_data2_path = os.path.join(data_path, 'causesofdeath20112022.csv')

# Load the data into a pandas DataFrame
df = pd.read_csv(death_data2_path)

# Display the first few rows of the DataFrame
df.head()

,DATAFLOW,LAST UPDATE,freq,unit,sex,age,icd10,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:HLTH_CD_ASDR2(1.0),18/10/23 23:00:00,A,RT,T,TOTAL,A-R_V-Y,AT,2011,994.58,NaN
1,ESTAT:HLTH_CD_ASDR2(1.0),18/10/23 23:00:00,A,RT,T,TOTAL,A-R_V-Y,AT,2012,1014.65,NaN
2,ESTAT:HLTH_CD_ASDR2(1.0),18/10/23 23:00:00,A,RT,T,TOTAL,A-R_V-Y,AT,2013,998.03,NaN
3,ESTAT:HLTH_CD_ASDR2(1.0),18/10/23 23:00:00,A,RT,T,TOTAL,A-R_V-Y,AT,2014,957.32,NaN
4,ESTAT:HLTH_CD_ASDR2(1.0),18/10/23 23:00:00,A,RT,T,TOTAL,A-R_V-Y,AT,2015,990.02,NaN


In [9]:
# Print the column names of the DataFrame
print(df.columns)

Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'sex', 'age', 'icd10', 'geo',
       'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG'],
      dtype='object')


In [10]:
# columns to delete
columns_to_delete = ["DATAFLOW", "LAST UPDATE", "freq", "unit", "sex", "age", "OBS_FLAG"]

# Remove the specified columns
df = df.drop(columns_to_delete, axis=1)

In [11]:
df.head()

,icd10,geo,TIME_PERIOD,OBS_VALUE
0,A-R_V-Y,AT,2011,994.58
1,A-R_V-Y,AT,2012,1014.65
2,A-R_V-Y,AT,2013,998.03
3,A-R_V-Y,AT,2014,957.32
4,A-R_V-Y,AT,2015,990.02


In [12]:
# Create a MultiIndex for rows with 'TIME_PERIOD' and 'geo'
rows_multiIndex = df.set_index(['TIME_PERIOD', 'geo'])

# Use MultiIndex to organize the data in the required format
df_final2 = rows_multiIndex.pivot_table(index=['TIME_PERIOD', 'geo'], columns=['icd10'], values='OBS_VALUE', aggfunc='mean', fill_value=0, dropna=False)

# Display the result
df_final2.head()

icd10            A-R_V-Y    ACC    A_B  C00-D48  D50-D89      E      F   F10  \
TIME_PERIOD geo                                                                
2011        AT    994.58  32.29   8.64   265.64     2.51  49.91  11.84  5.31   
            BE   1023.10  40.23  23.54   276.75     3.57  26.61  38.85  3.36   
            BG   1689.88  23.61   7.00   230.60     2.11  23.24   1.04  0.23   
            CH    872.60  34.10  10.93   237.80     2.51  21.97  62.79  3.01   
            CY   1024.77  30.23  16.28   202.09     8.94  83.95  18.42  0.34   

icd10              G_H        I       J      K     L     M      N  TOXICO  \
TIME_PERIOD geo                                                             
2011        AT   32.47   437.90   53.78  38.19  0.75  3.54  17.89    2.66   
            BE   45.52   311.37  104.95  45.40  2.72  4.92  25.61    0.27   
            BG   14.86  1181.53   61.37  45.07  0.34  0.94  17.14    0.10   
            CH   45.47   301.08   53.34  34.52  1.34  7.73  13.07    0.57   
            CY   29.26   415.73   79.08  38.38  7.74  4.17  35.28    0.00   

icd10            U071  V_Y85  X60-X84_Y870  X85-Y09_Y871  
TIME_PERIOD geo                                           
2011        AT    0.0   6.76         15.54          0.55  
            BE    0.0   8.54         19.27          1.35  
            BG    0.0   8.48         10.40          1.28  
            CH    0.0   4.64         13.23          0.53  
            CY    0.0   9.55          3.97          2.34

In [13]:
# Create a MultiIndex for columns with all unique codes for the main causes of death
columns_icd10 = df['icd10'].unique()
columns_multiIndex = pd.MultiIndex.from_product([columns_icd10], names=['causes of death'])

# Assign the MultiIndex to the DataFrame columns
df_final2.columns = columns_multiIndex

# Display the result
df_final2.head()

causes of death  A-R_V-Y    ACC    A_B C00-D48 D50-D89      E      F   F10  \
TIME_PERIOD geo                                                              
2011        AT    994.58  32.29   8.64  265.64    2.51  49.91  11.84  5.31   
            BE   1023.10  40.23  23.54  276.75    3.57  26.61  38.85  3.36   
            BG   1689.88  23.61   7.00  230.60    2.11  23.24   1.04  0.23   
            CH    872.60  34.10  10.93  237.80    2.51  21.97  62.79  3.01   
            CY   1024.77  30.23  16.28  202.09    8.94  83.95  18.42  0.34   

causes of death    G_H        I       J      K     L     M      N TOXICO U071  \
TIME_PERIOD geo                                                                 
2011        AT   32.47   437.90   53.78  38.19  0.75  3.54  17.89   2.66  0.0   
            BE   45.52   311.37  104.95  45.40  2.72  4.92  25.61   0.27  0.0   
            BG   14.86  1181.53   61.37  45.07  0.34  0.94  17.14   0.10  0.0   
            CH   45.47   301.08   53.34  34.52  1.34  7.73  13.07   0.57  0.0   
            CY   29.26   415.73   79.08  38.38  7.74  4.17  35.28   0.00  0.0   

causes of death V_Y85 X60-X84_Y870 X85-Y09_Y871  
TIME_PERIOD geo                                  
2011        AT   6.76        15.54         0.55  
            BE   8.54        19.27         1.35  
            BG   8.48        10.40         1.28  
            CH   4.64        13.23         0.53  
            CY   9.55         3.97         2.34

In [14]:
# Concatenate DataFrames along the rows
df_final = pd.concat([df_final1, df_final2], axis=0)

# Display the result
df_final.head()

causes of death A-R_V-Y   ACC   A_B C00-D48 D50-D89     E     F  F10   G_H  \
TIME_PERIOD geo                                                              
1994        AL      0.0   0.0   0.0     0.0     0.0   0.0   0.0  0.0   0.0   
            AT   1360.6  46.6   5.6   319.3     2.0  28.5   3.8  2.5  20.1   
            BE   1334.0  49.3  16.7   346.8     6.3  29.1  28.5  2.9  38.4   
            BG      0.0   0.0   0.0     0.0     0.0   0.0   0.0  0.0   0.0   
            CH   1146.3  52.7  17.0   306.8     3.9  30.0   6.0  2.0  23.1   

causes of death      I      J     K    L    M     N TOXICO U071 V_Y85  \
TIME_PERIOD geo                                                         
1994        AL     0.0    0.0   0.0  0.0  0.0   0.0    0.0  NaN   0.0   
            AT   748.1   60.8  64.2  0.7  2.4  14.1    0.7  NaN  17.0   
            BE   517.5  123.8  55.2  6.7  8.1  21.0    0.6  NaN  18.8   
            BG     0.0    0.0   0.0  0.0  0.0   0.0    0.0  NaN   0.0   
            CH   510.4   85.9  34.8  0.5  6.8  11.8    2.0  NaN  11.1   

causes of death X60-X84_Y870 X85-Y09_Y871  
TIME_PERIOD geo                            
1994        AL           0.0          0.0  
            AT          24.4          1.2  
            BE          22.5          1.8  
            BG           0.0          0.0  
            CH          23.0          1.3

In [15]:
# Rename both levels of the MultiIndex for rows
df_final.index.set_names(['year', 'country'], level=[0, 1], inplace=True)

# Display the DataFrame after modification
df_final.head()

causes of death A-R_V-Y   ACC   A_B C00-D48 D50-D89     E     F  F10   G_H  \
year country                                                                 
1994 AL             0.0   0.0   0.0     0.0     0.0   0.0   0.0  0.0   0.0   
     AT          1360.6  46.6   5.6   319.3     2.0  28.5   3.8  2.5  20.1   
     BE          1334.0  49.3  16.7   346.8     6.3  29.1  28.5  2.9  38.4   
     BG             0.0   0.0   0.0     0.0     0.0   0.0   0.0  0.0   0.0   
     CH          1146.3  52.7  17.0   306.8     3.9  30.0   6.0  2.0  23.1   

causes of death      I      J     K    L    M     N TOXICO U071 V_Y85  \
year country                                                            
1994 AL            0.0    0.0   0.0  0.0  0.0   0.0    0.0  NaN   0.0   
     AT          748.1   60.8  64.2  0.7  2.4  14.1    0.7  NaN  17.0   
     BE          517.5  123.8  55.2  6.7  8.1  21.0    0.6  NaN  18.8   
     BG            0.0    0.0   0.0  0.0  0.0   0.0    0.0  NaN   0.0   
     CH          510.4   85.9  34.8  0.5  6.8  11.8    2.0  NaN  11.1   

causes of death X60-X84_Y870 X85-Y09_Y871  
year country                               
1994 AL                  0.0          0.0  
     AT                 24.4          1.2  
     BE                 22.5          1.8  
     BG                  0.0          0.0  
     CH                 23.0          1.3

In [16]:
# Replace all 0.00 values with NaN in the combined DataFrame
df_final.replace(0.00, float('nan'), inplace=True)

I don't know how to format column names to appear on multiple lines for better visualization. Alternatively, we can avoid mapping the columns and keep the codes for each disease, specifying the legend in a note.

In [17]:
# Mapping the columns
column_mapping = {
    'A-R_V-Y': 'total deaths',
    'A_B': 'Infectious and parasitic diseases',
    'C00-D48' : 'Neoplasms',
    'D50-D89' : 'Diseases of the blood',
    'E' : 'Endocrine, nutritional and metabolic diseases',
    'F' : 'Mental and behavioural disorders',
    'F10' : 'Abuse of alcohol',
    'TOXICO' : 'Abuse of drugs',
    'G_H' : 'Diseases of the nervous system and the sense organs',
    'I' : 'Diseases of the circulatory system',
    'J' : 'Diseases of the respiratory system',
    'K' : 'Diseases of the digestive system',
    'L' : 'Diseases of the skin and subcutaneous tissue',
    'M' : 'Diseases of the musculoskeletal system and connective tissue',
    'N' : 'Diseases of the genitourinary system',
    'ACC' : 'Accidents',
    'V_Y85' : 'Transport accidents',
    'X60-X84_Y870' : 'Intentional self-harm',
    'X85-Y09_Y871' : 'Assault',
    'U071' : 'COVID-19',

}

# Change column names
df_final = df_final.rename(columns=column_mapping)

In [18]:
# Define the desired order of columns
desired_columns_order = ['total deaths', 'Neoplasms', 'Diseases of the circulatory system', 'Diseases of the respiratory system',
                         'Diseases of the nervous system and the sense organs', 'Endocrine, nutritional and metabolic diseases',
                         'Diseases of the digestive system', 'Infectious and parasitic diseases', 'Diseases of the blood',
                         'Mental and behavioural disorders', 'Abuse of alcohol', 'Abuse of drugs', 'Diseases of the skin and subcutaneous tissue',
                         'Diseases of the musculoskeletal system and connective tissue', 'Diseases of the genitourinary system',
                         'COVID-19', 'Accidents', 'Transport accidents', 'Intentional self-harm', 'Assault']

# Reorder columns based on the desired order
df_final = df_final[desired_columns_order]


In [19]:
df_final.head()

causes of death total deaths Neoplasms Diseases of the circulatory system  \
year country                                                                
1994 AL                  NaN       NaN                                NaN   
     AT               1360.6     319.3                              748.1   
     BE               1334.0     346.8                              517.5   
     BG                  NaN       NaN                                NaN   
     CH               1146.3     306.8                              510.4   

causes of death Diseases of the respiratory system  \
year country                                         
1994 AL                                        NaN   
     AT                                       60.8   
     BE                                      123.8   
     BG                                        NaN   
     CH                                       85.9   

causes of death Diseases of the nervous system and the sense organs  \
year country                                                          
1994 AL                                                        NaN    
     AT                                                       20.1    
     BE                                                       38.4    
     BG                                                        NaN    
     CH                                                       23.1    

causes of death Endocrine, nutritional and metabolic diseases  \
year country                                                    
1994 AL                                                   NaN   
     AT                                                  28.5   
     BE                                                  29.1   
     BG                                                   NaN   
     CH                                                  30.0   

causes of death Diseases of the digestive system  \
year country                                       
1994 AL                                      NaN   
     AT                                     64.2   
     BE                                     55.2   
     BG                                      NaN   
     CH                                     34.8   

causes of death Infectious and parasitic diseases Diseases of the blood  \
year country                                                              
1994 AL                                       NaN                   NaN   
     AT                                       5.6                   2.0   
     BE                                      16.7                   6.3   
     BG                                       NaN                   NaN   
     CH                                      17.0                   3.9   

causes of death Mental and behavioural disorders Abuse of alcohol  \
year country                                                        
1994 AL                                      NaN              NaN   
     AT                                      3.8              2.5   
     BE                                     28.5              2.9   
     BG                                      NaN              NaN   
     CH                                      6.0              2.0   

causes of death Abuse of drugs Diseases of the skin and subcutaneous tissue  \
year country                                                                  
1994 AL                    NaN                                          NaN   
     AT                    0.7                                          0.7   
     BE                    0.6                                          6.7   
     BG                    NaN                                          NaN   
     CH                    2.0                                          0.5   

causes of death Diseases of the musculoskeletal system and connective tissue  \
year country                                                                   
1994 AL                               

I can't map the countries correctly because they are not unique elements in the 'geo' column



In [20]:
# Mapping the rows
row_mapping = {
    'AL': 'Albania',
    'AT': 'Austria',
    'BE': 'Belgium',
    'BG': 'Bulgaria',
    'CH': 'Switzerland',
    'CY': 'Cyprus',
    'CZ': 'Czech Republic',
    'DE': 'Germany',
    'DK': 'Denmark',
    'EE': 'Estonia',
    'EL': 'Greece',
    'ES': 'Spain',
    'FI': 'Finland',
    'FR': 'France',
    'HR': 'Croatia',
    'HU': 'Hungary',
    'IE': 'Ireland',
    'IS': 'Iceland',
    'IT': 'Italy',
    'LI': 'Liechtenstein',
    'LT': 'Lithuania',
    'LU': 'Luxembourg',
    'LV': 'Latvia',
    'MK': 'North Macedonia',
    'MT': 'Malta',
    'NL': 'Netherlands',
    'NO': 'Norway',
    'PL': 'Poland',
    'PT': 'Portugal',
    'RO': 'Romania',
    'RS': 'Serbia',
    'SE': 'Sweden',
    'SI': 'Slovenia',
    'SK': 'Slovakia',
    'TR': 'Turkey',
    'UK': 'United Kingdom',
    'EU28' : 'EU country (2013/2020)',
    'EU27_2020' : 'EU country (from 2020)'
}

# Apply the mapping to the 'geo\TIME_PERIOD' level
df_final.index = df_final.index.set_levels(df_final.index.levels[1].map(row_mapping.get), level=1)

# Display the DataFrame after modification
df_final

causes of death     total deaths Neoplasms Diseases of the circulatory system  \
year country                                                                    
1994 Albania                 NaN       NaN                                NaN   
     Austria             1360.60    319.30                             748.10   
     Belgium             1334.00    346.80                             517.50   
     Bulgaria                NaN       NaN                                NaN   
     Switzerland         1146.30    306.80                             510.40   
...                          ...       ...                                ...   
2020 Sweden               934.38    223.36                             267.30   
     Slovenia            1112.42    293.55                             377.11   
     Slovakia            1365.47    303.26                             607.09   
     Turkey                  NaN       NaN                                NaN   
     United Kingdom          NaN       NaN                                NaN   

causes of death     Diseases of the respiratory system  \
year country                                             
1994 Albania                                       NaN   
     Austria                                     60.80   
     Belgium                                    123.80   
     Bulgaria                                      NaN   
     Switzerland                                 85.90   
...                                                ...   
2020 Sweden                                      52.20   
     Slovenia                                    38.59   
     Slovakia                                    94.86   
     Turkey                                        NaN   
     United Kingdom                                NaN   

causes of death     Diseases of the nervous system and the sense organs  \
year country                                                              
1994 Albania                                                       NaN    
     Austria                                                     20.10    
     Belgium                                                     38.40    
     Bulgaria                                                      NaN    
     Switzerland                                                 23.10    
...                                                                ...    
2020 Sweden                                                      56.44    
     Slovenia                                                    44.03    
     Slovakia                                                    26.91    
     Turkey                                                        NaN    
     United Kingdom                                                NaN    

causes of death     Endocrine, nutritional and metabolic diseases  \
year country                                                        
1994 Albania                                                  NaN   
     Austria                                                28.50   
     Belgium                                                29.10   
     Bulgaria                                                 NaN   
     Switzerland                                            30.00   
...                                                           ...   
2020 Sweden                                                 28.83   
     Slovenia                                               24.40   
     Slovakia                                               21.23   
     Turkey                                                   NaN   
     United Kingdom                                           NaN   

causes of death     Diseases of the digestive system  \
year country                                           
1994 Albania                                     NaN   
     Austria                                   64.20   
     Belgium                                   55.20   
     Bulgaria                        

Loading relevant data about health of the population for each country.
All the eurostat dataset on this topic are refered only to two years (2014 and 2019). We only have two years of reference, but it is still possible to assume a trend.


In [21]:
# Create the path for the dataset
alcohol_data_path = os.path.join(data_path, 'alcohol_consumption.csv')
tobacco_data_path = os.path.join(data_path, 'tobacco_consumption.csv')
physica_activity_data_path = os.path.join(data_path, 'physical_activity.csv')
nutrition_data_path = os.path.join(data_path, 'fruit_and_veg_consumption.csv')
physical_condition_path = os.path.join(data_path, 'body_mass_index.csv')

# Load the data into a pandas DataFrame
df1 = pd.read_csv(alcohol_data_path)
df2 = pd.read_csv(tobacco_data_path)
df3 = pd.read_csv(physica_activity_data_path)
df4 = pd.read_csv(nutrition_data_path)
df5 = pd.read_csv(physical_condition_path)

In [22]:
df1["TIME_PERIOD"].unique()

array([2014, 2019], dtype=int64)

In [23]:
# Display the first few rows of the DataFrame
display(df1.head())
display(df2.head())
display(df3.head())
display(df4.head())
display(df5.head())


print(max(df1["TIME_PERIOD"]))

,DATAFLOW,LAST UPDATE,freq,unit,frequenc,sex,age,citizen,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:HLTH_EHIS_AL1C(1.0),08/11/22 23:00:00,A,PC,DAY,T,TOTAL,NAT,AT,2014,6.5,NaN
1,ESTAT:HLTH_EHIS_AL1C(1.0),08/11/22 23:00:00,A,PC,DAY,T,TOTAL,NAT,AT,2019,6.1,NaN
2,ESTAT:HLTH_EHIS_AL1C(1.0),08/11/22 23:00:00,A,PC,DAY,T,TOTAL,NAT,BE,2014,14.5,NaN
3,ESTAT:HLTH_EHIS_AL1C(1.0),08/11/22 23:00:00,A,PC,DAY,T,TOTAL,NAT,BE,2019,10.0,NaN
4,ESTAT:HLTH_EHIS_AL1C(1.0),08/11/22 23:00:00,A,PC,DAY,T,TOTAL,NAT,BG,2014,8.8,NaN


,DATAFLOW,LAST UPDATE,freq,unit,smoking,sex,age,citizen,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:HLTH_EHIS_SK1C(1.0),08/11/22 23:00:00,A,PC,SM_DAY,T,TOTAL,NAT,AT,2014,22.4,NaN
1,ESTAT:HLTH_EHIS_SK1C(1.0),08/11/22 23:00:00,A,PC,SM_DAY,T,TOTAL,NAT,AT,2019,19.1,NaN
2,ESTAT:HLTH_EHIS_SK1C(1.0),08/11/22 23:00:00,A,PC,SM_DAY,T,TOTAL,NAT,BE,2014,18.7,u
3,ESTAT:HLTH_EHIS_SK1C(1.0),08/11/22 23:00:00,A,PC,SM_DAY,T,TOTAL,NAT,BE,2019,15.2,NaN
4,ESTAT:HLTH_EHIS_SK1C(1.0),08/11/22 23:00:00,A,PC,SM_DAY,T,TOTAL,NAT,BG,2014,28.2,NaN


,DATAFLOW,LAST UPDATE,freq,unit,physact,sex,age,citizen,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:HLTH_EHIS_PE9C(1.0),01/08/23 23:00:00,A,PC,MV_AERO_MSC,T,TOTAL,NAT,AT,2014,25.1,NaN
1,ESTAT:HLTH_EHIS_PE9C(1.0),01/08/23 23:00:00,A,PC,MV_AERO_MSC,T,TOTAL,NAT,AT,2019,22.4,NaN
2,ESTAT:HLTH_EHIS_PE9C(1.0),01/08/23 23:00:00,A,PC,MV_AERO_MSC,T,TOTAL,NAT,BE,2019,5.7,NaN
3,ESTAT:HLTH_EHIS_PE9C(1.0),01/08/23 23:00:00,A,PC,MV_AERO_MSC,T,TOTAL,NAT,BG,2014,5.3,NaN
4,ESTAT:HLTH_EHIS_PE9C(1.0),01/08/23 23:00:00,A,PC,MV_AERO_MSC,T,TOTAL,NAT,BG,2019,5.8,NaN


,DATAFLOW,LAST UPDATE,freq,unit,n_portion,sex,age,citizen,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:HLTH_EHIS_FV3C(1.0),08/11/22 23:00:00,A,PC,0,T,TOTAL,NAT,AT,2014,32.2,NaN
1,ESTAT:HLTH_EHIS_FV3C(1.0),08/11/22 23:00:00,A,PC,0,T,TOTAL,NAT,AT,2019,36.5,NaN
2,ESTAT:HLTH_EHIS_FV3C(1.0),08/11/22 23:00:00,A,PC,0,T,TOTAL,NAT,BE,2014,15.2,NaN
3,ESTAT:HLTH_EHIS_FV3C(1.0),08/11/22 23:00:00,A,PC,0,T,TOTAL,NAT,BE,2019,16.6,NaN
4,ESTAT:HLTH_EHIS_FV3C(1.0),08/11/22 23:00:00,A,PC,0,T,TOTAL,NAT,BG,2014,58.7,NaN


,DATAFLOW,LAST UPDATE,freq,unit,bmi,sex,age,citizen,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:HLTH_EHIS_BM1C(1.0),08/11/22 23:00:00,A,PC,BMI18P5-24,T,TOTAL,NAT,AT,2014,49.5,NaN
1,ESTAT:HLTH_EHIS_BM1C(1.0),08/11/22 23:00:00,A,PC,BMI18P5-24,T,TOTAL,NAT,AT,2019,46.1,NaN
2,ESTAT:HLTH_EHIS_BM1C(1.0),08/11/22 23:00:00,A,PC,BMI18P5-24,T,TOTAL,NAT,BE,2014,48.5,NaN
3,ESTAT:HLTH_EHIS_BM1C(1.0),08/11/22 23:00:00,A,PC,BMI18P5-24,T,TOTAL,NAT,BE,2019,48.5,NaN
4,ESTAT:HLTH_EHIS_BM1C(1.0),08/11/22 23:00:00,A,PC,BMI18P5-24,T,TOTAL,NAT,BG,2014,44.6,NaN


2019


In [24]:
# Print the column names of the DataFrame
display(df1.columns)
display(df2.columns)
display(df3.columns)
display(df4.columns)
display(df5.columns)

Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'frequenc', 'sex', 'age',
       'citizen', 'geo', 'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG'],
      dtype='object')

Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'smoking', 'sex', 'age',
       'citizen', 'geo', 'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG'],
      dtype='object')

Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'physact', 'sex', 'age',
       'citizen', 'geo', 'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG'],
      dtype='object')

Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'n_portion', 'sex', 'age',
       'citizen', 'geo', 'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG'],
      dtype='object')

Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'unit', 'bmi', 'sex', 'age',
       'citizen', 'geo', 'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG'],
      dtype='object')

In [25]:
# Preserve only data about daily drinkers
df1 = df1[df1['frequenc'] != 'WEEK']
df1 = df1[df1['frequenc'] != 'NVR']

# Delete data about regular consumers of fruits/vegetables
df4 = df4[df4['n_portion'] != 'GE5']

# Preserve only data about overweight population
df5 = df5[df5['bmi'] != 'BMI18P5-24']
df5 = df5[df5['bmi'] != 'BMI_LT18P5']
df5 = df5[df5['bmi'] != 'BMI_GE30']

# Reset the row numbering starting from 1
df5.reset_index(drop=True, inplace=True)

In [26]:
# columns to delete
columns_to_delete1 = ["DATAFLOW", "LAST UPDATE", "freq", "unit", "frequenc", "sex", "age", "citizen", "OBS_FLAG"]
columns_to_delete2 = ["DATAFLOW", "LAST UPDATE", "freq", "unit", "smoking", "sex", "age", "citizen", "OBS_FLAG"]
columns_to_delete3 = ["DATAFLOW", "LAST UPDATE", "freq", "unit", "physact", "sex", "age", "citizen", "OBS_FLAG"]
columns_to_delete4 = ["DATAFLOW", "LAST UPDATE", "freq", "unit", "n_portion", "sex", "age", "citizen", "OBS_FLAG"]
columns_to_delete5 = ["DATAFLOW", "LAST UPDATE", "freq", "unit", "sex", "bmi", "age", "citizen", "OBS_FLAG"]

# Remove the specified columns
df1 = df1.drop(columns_to_delete1, axis=1)
df2 = df2.drop(columns_to_delete2, axis=1)
df3 = df3.drop(columns_to_delete3, axis=1)
df4 = df4.drop(columns_to_delete4, axis=1)
df5 = df5.drop(columns_to_delete5, axis=1)

In [27]:
df1.rename(columns={'OBS_VALUE': '% daily Alcohol consumers'}, inplace=True)
df2.rename(columns={'OBS_VALUE': '% daily Tobacco consumers'}, inplace=True)
df3.rename(columns={'OBS_VALUE': '% active population'}, inplace=True)
df4.rename(columns={'OBS_VALUE': '% no daily Fruit/Veg consumers'}, inplace=True)
df5.rename(columns={'OBS_VALUE': '% overweight population'}, inplace=True)

In [28]:
# Fill missing rows
def fill_missing_rows(df):

    df_filled = df.copy()

    # Check for missing rows and add them
    for geo_value in df['geo'].unique():

        if len(df[(df['geo'] == geo_value) & (df['TIME_PERIOD'] == 2014)]) == 0:

            # Add row for 2014
            row_2014 = pd.Series({'geo': geo_value, 'TIME_PERIOD': 2014})
            df = pd.concat([df, row_2014.to_frame().T], ignore_index=True)

        if len(df[(df['geo'] == geo_value) & (df['TIME_PERIOD'] == 2019)]) == 0:

            # Add row for 2019
            row_2019 = pd.Series({'geo': geo_value, 'TIME_PERIOD': 2019})
            df = pd.concat([df, row_2019.to_frame().T], ignore_index=True)

    # Sort the DataFrame by 'geo' and 'TIME_PERIOD'
    df = df.sort_values(by=['geo', 'TIME_PERIOD']).reset_index(drop=True)

    return df

# df1, df2, df3, df4, df5 are the DataFrames to fill
df1 = fill_missing_rows(df1)
df2 = fill_missing_rows(df2)
df3 = fill_missing_rows(df3)
df4 = fill_missing_rows(df4)
df5 = fill_missing_rows(df5)

# Display the filled DataFrames
display(df1)
display(df2)
display(df3)
display(df4)
display(df5)

,geo,TIME_PERIOD,% daily Alcohol consumers
0,AT,2014,6.5
1,AT,2019,6.1
2,BE,2014,14.5
3,BE,2019,10.0
4,BG,2014,8.8
...,...,...,...
61,SK,2019,4.1
62,TR,2014,0.5
63,TR,2019,NaN
64,UK,2014,7.8


,geo,TIME_PERIOD,% daily Tobacco consumers
0,AT,2014,22.4
1,AT,2019,19.1
2,BE,2014,18.7
3,BE,2019,15.2
4,BG,2014,28.2
...,...,...,...
61,SK,2019,21.0
62,TR,2014,27.3
63,TR,2019,NaN
64,UK,2014,13.9


,geo,TIME_PERIOD,% active population
0,AT,2014,25.1
1,AT,2019,22.4
2,BE,2014,NaN
3,BE,2019,5.7
4,BG,2014,5.3
...,...,...,...
61,SK,2019,9.9
62,TR,2014,1.2
63,TR,2019,NaN
64,UK,2014,13.8


,geo,TIME_PERIOD,% no daily Fruit/Veg consumers
0,AT,2014,32.2
1,AT,2019,36.5
2,BE,2014,15.2
3,BE,2019,16.6
4,BG,2014,58.7
...,...,...,...
61,SK,2019,41.3
62,TR,2014,33.8
63,TR,2019,NaN
64,UK,2014,21.4


,geo,TIME_PERIOD,% overweight population
0,AT,2014,48.0
1,AT,2019,51.4
2,BE,2014,48.3
3,BE,2019,48.3
4,BG,2014,52.9
...,...,...,...
61,SK,2019,57.9
62,TR,2014,53.6
63,TR,2019,NaN
64,UK,2014,55.6


In [29]:
# from matplotlib import pyplot as plt
# df5['% overweight population'].plot(kind='line', figsize=(8, 4), title='% overweight population')
# plt.gca().spines[['top', 'right']].set_visible(False)

In [30]:
# from matplotlib import pyplot as plt
# df5['% overweight population'].plot(kind='hist', bins=20, title='% overweight population')
# plt.gca().spines[['top', 'right',]].set_visible(False)

In [31]:
# Select only the unique columns from each DataFrame
df1_unique_columns = df1[['geo', 'TIME_PERIOD'] + list(set(df1.columns) - {'geo', 'TIME_PERIOD'})]
df2_unique_columns = df2[list(set(df2.columns) - {'geo', 'TIME_PERIOD'})]
df3_unique_columns = df3[list(set(df3.columns) - {'geo', 'TIME_PERIOD'})]
df4_unique_columns = df4[list(set(df4.columns) - {'geo', 'TIME_PERIOD'})]
df5_unique_columns = df5[list(set(df5.columns) - {'geo', 'TIME_PERIOD'})]

# Concatenate DataFrames along the columns
df_health = pd.concat([df1_unique_columns, df2_unique_columns, df3_unique_columns, df4_unique_columns, df5_unique_columns], axis=1)

# Display the result
df_health.head()

,geo,TIME_PERIOD,% daily Alcohol consumers,% daily Tobacco consumers,% active population,% no daily Fruit/Veg consumers,% overweight population
0,AT,2014,6.5,22.4,25.1,32.2,48.0
1,AT,2019,6.1,19.1,22.4,36.5,51.4
2,BE,2014,14.5,18.7,NaN,15.2,48.3
3,BE,2019,10.0,15.2,5.7,16.6,48.3
4,BG,2014,8.8,28.2,5.3,58.7,52.9


In [32]:
df_health.rename(columns={'geo': 'country', 'TIME_PERIOD': 'year'}, inplace=True)

Loading relevant demographic data for each country.
This data are completed from 1994 to 2022

In [33]:
# Create the path for the dataset
population_structure_path = os.path.join(data_path, 'population_structure.csv')
demographic_rates_path = os.path.join(data_path, 'demographic_rates.csv')

# Load the data into a pandas DataFrame
df_pop_structure = pd.read_csv(population_structure_path)
df_dem_rates = pd.read_csv(demographic_rates_path)

# Display the first few rows of the DataFrame
display(df_pop_structure.head())
df_dem_rates.head()

,DATAFLOW,LAST UPDATE,freq,indic_de,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:DEMO_PJANIND(1.0),28/09/23 23:00:00,A,MEDAGEPOP,AL,2002,27.6,NaN
1,ESTAT:DEMO_PJANIND(1.0),28/09/23 23:00:00,A,MEDAGEPOP,AL,2003,27.9,NaN
2,ESTAT:DEMO_PJANIND(1.0),28/09/23 23:00:00,A,MEDAGEPOP,AL,2004,28.2,NaN
3,ESTAT:DEMO_PJANIND(1.0),28/09/23 23:00:00,A,MEDAGEPOP,AL,2005,28.7,NaN
4,ESTAT:DEMO_PJANIND(1.0),28/09/23 23:00:00,A,MEDAGEPOP,AL,2006,29.2,NaN


,DATAFLOW,LAST UPDATE,freq,indic_de,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,ESTAT:DEMO_GIND(1.0),27/11/23 23:00:00,A,AVG,AL,1994,3187784,NaN
1,ESTAT:DEMO_GIND(1.0),27/11/23 23:00:00,A,AVG,AL,1995,3168033,NaN
2,ESTAT:DEMO_GIND(1.0),27/11/23 23:00:00,A,AVG,AL,1996,3148281,NaN
3,ESTAT:DEMO_GIND(1.0),27/11/23 23:00:00,A,AVG,AL,1997,3128530,NaN
4,ESTAT:DEMO_GIND(1.0),27/11/23 23:00:00,A,AVG,AL,1998,3108778,NaN


In [34]:
# Print the column names of the DataFrame
print(df_pop_structure.columns)
print(df_dem_rates.columns)

Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'indic_de', 'geo', 'TIME_PERIOD',
       'OBS_VALUE', 'OBS_FLAG'],
      dtype='object')
Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'indic_de', 'geo', 'TIME_PERIOD',
       'OBS_VALUE', 'OBS_FLAG'],
      dtype='object')


In [35]:
# Columns to delete
columns_to_delete = ["DATAFLOW", "LAST UPDATE", "freq", "OBS_FLAG"]

# Remove the specified columns
df_dem_rates = df_dem_rates.drop(columns_to_delete, axis=1)
df_pop_structure = df_pop_structure.drop(columns_to_delete, axis=1)

In [36]:
# Create a MultiIndex for rows with 'TIME_PERIOD' and 'geo'
rows_multiIndex1 = df_dem_rates.set_index(['TIME_PERIOD', 'geo'])
rows_multiIndex2 = df_pop_structure.set_index(['TIME_PERIOD', 'geo'])

# Use MultiIndex to organize the data in the required format
df_dem_rates = rows_multiIndex1.pivot_table(index=['TIME_PERIOD', 'geo'], columns=['indic_de'], values='OBS_VALUE', aggfunc='mean', fill_value=0, dropna=False)
df_pop_structure = rows_multiIndex2.pivot_table(index=['TIME_PERIOD', 'geo'], columns=['indic_de'], values='OBS_VALUE', aggfunc='mean', fill_value=0, dropna=False)

# Display the result
print(df_dem_rates)
print(df_pop_structure)

indic_de              AVG  CNMIGRAT
TIME_PERIOD geo                    
1994        AL    3187784    -29339
            AT    7936118      3012
            BE   10115603     17208
            BG    8443591        40
            CH    6993795     29457
...                   ...       ...
2022        SE   10486941     59233
            SI    2111986     14477
            SK    5431752       995
            TR   84979913     68324
            UK          0         0

[1044 rows x 2 columns]
indic_de         MEDAGEPOP  PC_Y60_MAX  PC_Y80_MAX
TIME_PERIOD geo                                   
1994        AL         0.0         0.0         0.0
            AT        35.9        19.8         3.8
            BE        37.0        21.2         3.7
            BG        37.8        20.8         2.4
            CH        37.0        19.4         3.9
...                    ...         ...         ...
2022        SE        40.7        25.7         5.4
            SI        44.7        27.9         5

In [37]:
# Remove the name of the 'indic_de' level for the columns
df_dem_rates.columns = df_dem_rates.columns.set_names([None])
df_pop_structure.columns = df_pop_structure.columns.set_names([None])

# Replace column names with new names
df_dem_rates.rename(columns={'AVG': 'total population', 'CNMIGRAT': 'Immigrants - emigrants'}, inplace=True)
df_pop_structure.rename(columns={'MEDAGEPOP': 'median age', 'PC_Y60_MAX': 'over 60', 'PC_Y80_MAX': 'over 80'}, inplace=True)

# Print the DataFrame with the new column names
print(df_dem_rates)
print(df_pop_structure)

                 total population  Immigrants - emigrants
TIME_PERIOD geo                                          
1994        AL            3187784                  -29339
            AT            7936118                    3012
            BE           10115603                   17208
            BG            8443591                      40
            CH            6993795                   29457
...                           ...                     ...
2022        SE           10486941                   59233
            SI            2111986                   14477
            SK            5431752                     995
            TR           84979913                   68324
            UK                  0                       0

[1044 rows x 2 columns]
                 median age  over 60  over 80
TIME_PERIOD geo                              
1994        AL          0.0      0.0      0.0
            AT         35.9     19.8      3.8
            BE         37.0     21.2 

In [38]:
# Reset index for both DataFrames
df_dem_rates_reset = df_dem_rates.reset_index()
df_pop_structure_reset = df_pop_structure.reset_index()

# Select only the unique columns from each DataFrame
df_dem_rates_unique_columns = df_dem_rates_reset[['geo', 'TIME_PERIOD'] + list(set(df_dem_rates_reset.columns) - {'geo', 'TIME_PERIOD'})]
df_pop_structure_unique_columns = df_pop_structure_reset[list(set(df_pop_structure_reset.columns) - {'geo', 'TIME_PERIOD'})]

# Concatenate DataFrames along the columns
df_demographic = pd.concat([df_dem_rates_unique_columns, df_pop_structure_unique_columns], axis=1)

df_demographic.rename(columns={'geo': 'country', 'TIME_PERIOD': 'year'}, inplace=True)

# Display the resulting DataFrame
print(df_demographic)

     country  year  Immigrants - emigrants  total population  median age  \
0         AL  1994                  -29339           3187784         0.0   
1         AT  1994                    3012           7936118        35.9   
2         BE  1994                   17208          10115603        37.0   
3         BG  1994                      40           8443591        37.8   
4         CH  1994                   29457           6993795        37.0   
...      ...   ...                     ...               ...         ...   
1039      SE  2022                   59233          10486941        40.7   
1040      SI  2022                   14477           2111986        44.7   
1041      SK  2022                     995           5431752        41.8   
1042      TR  2022                   68324          84979913         0.0   
1043      UK  2022                       0                 0         0.0   

      over 60  over 80  
0         0.0      0.0  
1        19.8      3.8  
2        21.

In [39]:
# Replace all 0.0 values with NaN in the combined DataFrame
df_demographic.replace(0.0, float('nan'), inplace=True)

# Display the resulting DataFrame
df_demographic.head()

,country,year,Immigrants - emigrants,total population,median age,over 60,over 80
0,AL,1994,-29339.0,3187784.0,NaN,NaN,NaN
1,AT,1994,3012.0,7936118.0,35.9,19.8,3.8
2,BE,1994,17208.0,10115603.0,37.0,21.2,3.7
3,BG,1994,40.0,8443591.0,37.8,20.8,2.4
4,CH,1994,29457.0,6993795.0,37.0,19.4,3.9


In [40]:
# Saving the processed dataset so we can load it in
df_demographic.to_csv("DemographicData.csv",index=True)
df_final1.to_csv("df_final1.csv",index=True)
df_final2.to_csv("df_final2.csv",index=True)
df_final.to_csv("df_final.csv",index=True)
df_health.to_csv("df_health.csv",index=True)


In [41]:
df_final

causes of death     total deaths Neoplasms Diseases of the circulatory system  \
year country                                                                    
1994 Albania                 NaN       NaN                                NaN   
     Austria             1360.60    319.30                             748.10   
     Belgium             1334.00    346.80                             517.50   
     Bulgaria                NaN       NaN                                NaN   
     Switzerland         1146.30    306.80                             510.40   
...                          ...       ...                                ...   
2020 Sweden               934.38    223.36                             267.30   
     Slovenia            1112.42    293.55                             377.11   
     Slovakia            1365.47    303.26                             607.09   
     Turkey                  NaN       NaN                                NaN   
     United Kingdom          NaN       NaN                                NaN   

causes of death     Diseases of the respiratory system  \
year country                                             
1994 Albania                                       NaN   
     Austria                                     60.80   
     Belgium                                    123.80   
     Bulgaria                                      NaN   
     Switzerland                                 85.90   
...                                                ...   
2020 Sweden                                      52.20   
     Slovenia                                    38.59   
     Slovakia                                    94.86   
     Turkey                                        NaN   
     United Kingdom                                NaN   

causes of death     Diseases of the nervous system and the sense organs  \
year country                                                              
1994 Albania                                                       NaN    
     Austria                                                     20.10    
     Belgium                                                     38.40    
     Bulgaria                                                      NaN    
     Switzerland                                                 23.10    
...                                                                ...    
2020 Sweden                                                      56.44    
     Slovenia                                                    44.03    
     Slovakia                                                    26.91    
     Turkey                                                        NaN    
     United Kingdom                                                NaN    

causes of death     Endocrine, nutritional and metabolic diseases  \
year country                                                        
1994 Albania                                                  NaN   
     Austria                                                28.50   
     Belgium                                                29.10   
     Bulgaria                                                 NaN   
     Switzerland                                            30.00   
...                                                           ...   
2020 Sweden                                                 28.83   
     Slovenia                                               24.40   
     Slovakia                                               21.23   
     Turkey                                                   NaN   
     United Kingdom                                           NaN   

causes of death     Diseases of the digestive system  \
year country                                           
1994 Albania                                     NaN   
     Austria                                   64.20   
     Belgium                                   55.20   
     Bulgaria                        

In [42]:
df_final1

causes of death A-R_V-Y   ACC   A_B C00-D48 D50-D89     E     F  F10   G_H  \
TIME_PERIOD geo                                                              
1994        AL      0.0   0.0   0.0     0.0     0.0   0.0   0.0  0.0   0.0   
            AT   1360.6  46.6   5.6   319.3     2.0  28.5   3.8  2.5  20.1   
            BE   1334.0  49.3  16.7   346.8     6.3  29.1  28.5  2.9  38.4   
            BG      0.0   0.0   0.0     0.0     0.0   0.0   0.0  0.0   0.0   
            CH   1146.3  52.7  17.0   306.8     3.9  30.0   6.0  2.0  23.1   
...                 ...   ...   ...     ...     ...   ...   ...  ...   ...   
2010        RO   1585.9  44.8  12.9   262.7     0.8  14.8   1.5  1.0  15.5   
            SE    972.7  31.6  20.8   248.6     3.1  26.2  53.0  2.9  39.0   
            SI   1083.6  51.5   4.9   326.9     2.1  16.4   7.2  6.9  16.9   
            SK   1507.4  35.9   9.9   310.4     1.2  20.3   0.0  0.0  18.6   
            UK   1016.3  23.2  10.7   294.0     2.0  15.3  42.7  1.4  37.6   

causes of death       I      J     K    L    M     N TOXICO V_Y85  \
TIME_PERIOD geo                                                     
1994        AL      0.0    0.0   0.0  0.0  0.0   0.0    0.0   0.0   
            AT    748.1   60.8  64.2  0.7  2.4  14.1    0.7  17.0   
            BE    517.5  123.8  55.2  6.7  8.1  21.0    0.6  18.8   
            BG      0.0    0.0   0.0  0.0  0.0   0.0    0.0   0.0   
            CH    510.4   85.9  34.8  0.5  6.8  11.8    2.0  11.1   
...                 ...    ...   ...  ...  ...   ...    ...   ...   
2010        RO   1016.3   75.5  89.9  0.5  0.2  15.1    0.0  14.1   
            SE    381.3   54.7  29.4  1.6  5.1  11.9    0.3   3.4   
            SI    451.5   69.5  63.8  2.8  4.5  16.8    0.0   8.3   
            SK    877.9   97.1  67.4  0.0  1.4  22.1    0.0   9.6   
            UK    325.9  138.8  52.8  3.7  8.4  25.6    1.5   3.6   

causes of death X60-X84_Y870 X85-Y09_Y871  
TIME_PERIOD geo                            
1994        AL           0.0          0.0  
            AT          24.4          1.2  
            BE          22.5          1.8  
            BG           0.0          0.0  
            CH          23.0          1.3  
...                      ...          ...  
2010        RO          13.7          2.6  
            SE          12.4          0.9  
            SI          20.3          0.5  
            SK          12.0          1.3  
            UK           6.8          0.3  

[595 rows x 19 columns]

In [43]:
column_names = df_final.columns
column_names_list = column_names.tolist()
print(column_names_list)
country_names = df_final.index.get_level_values('country').unique()
country_names_list = country_names.tolist()
print(country_names_list)
years = df_final.index.get_level_values('year').unique()
years_list = years.tolist()
print(years_list)

[('total deaths',), ('Neoplasms',), ('Diseases of the circulatory system',), ('Diseases of the respiratory system',), ('Diseases of the nervous system and the sense organs',), ('Endocrine, nutritional and metabolic diseases',), ('Diseases of the digestive system',), ('Infectious and parasitic diseases',), ('Diseases of the blood',), ('Mental and behavioural disorders',), ('Abuse of alcohol',), ('Abuse of drugs',), ('Diseases of the skin and subcutaneous tissue',), ('Diseases of the musculoskeletal system and connective tissue',), ('Diseases of the genitourinary system',), ('COVID-19',), ('Accidents',), ('Transport accidents',), ('Intentional self-harm',), ('Assault',)]
['Albania', 'Austria', 'Belgium', 'Bulgaria', 'Switzerland', 'Cyprus', 'Czech Republic', 'Germany', 'Denmark', 'Estonia', 'Greece', 'Spain', 'EU country (2013/2020)', 'Finland', 'France', 'Croatia', 'Hungary', 'Ireland', 'Iceland', 'Italy', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Latvia', 'North Macedonia', 'Malta',

# Data preparation
- Rows, where no data is given (no deaths for any cause is known), are simply dropped. These countries will not be used for the years where they have no data.
- All other rows with just some missing columns were filled up by the mean values of the other countries of the same year
- The deaths were changed to percentages of the total deaths. Just the column "total deaths" is kept as it is , since this can also be used for comparison

In [44]:
df_final = df_final.dropna(axis = 0, how = 'all') # drop empty rows

def fillna_with_yearly_column_mean(column):
    return column.fillna(column.groupby('year').transform('mean'))

# Apply the fillna_with_yearly_column_mean function to each column
df_final_filled = df_final.apply(fillna_with_yearly_column_mean, axis=0)
df_final_filled = df_final_filled.fillna(0)
total_deaths = df_final_filled["total deaths"].values

# Divide all causes by "total deaths"
df_percent = df_final_filled.div(total_deaths)
df_percent["total deaths"] = df_final_filled["total deaths"]

# Clustering
The countries are always just compared per year - the code would need to be changed to compare over all years as well
To compare the countries, I used
1. KMeans Clustering: All columns were used to compare countries and assign to k different clusters
2. JenksNaturalBreaks: Every column was compared individually and based on that the countries were assigned to different groups.

Why not use KMeans for both? Apparently for 1D data (1 column) there are better ways to group data, than clustering. One good metric is JenksNaturalBreaks: https://en.wikipedia.org/wiki/Jenks_natural_breaks_optimization
It seeks to minimize each class's average deviation from the class mean, while maximizing each class's deviation from the means of the other classes.

For 1. we can already take the resulting clusters to make asumptions on which countries are simillar.
For 2. we need to compare all the classes for each column to group the countries

The resulting clusters/classes are stored in the df_clusters


In [45]:
from sklearn.cluster import KMeans
from jenkspy import JenksNaturalBreaks

def warn(*args, **kwargs): # for disabling the stupid warnings
    pass
import warnings
warnings.warn = warn

data = {
    'Cluster': [0]
}
df_clusters = pd.DataFrame(data, index=df_percent.index)
k=10 # how many clusters/intervals we want
kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
jnb = JenksNaturalBreaks(k)
for year in df_percent.index.get_level_values('year').unique():
    df_percent_year = df_percent.loc[year]
    for column in df_percent.columns:
        if (column[0] == "COVID-19" and year != 2020): # because Covid deaths only start at 2020, before there are no measurements
            df_clusters.loc[(year, slice(None)), column[0]+" cluster"] = [0 for j in range(len(df_percent_year[column]))]
        else:
            jnb.fit(df_percent_year[column])
            df_clusters.loc[(year, slice(None)), column[0]+" cluster"] =  jnb.labels_
    df_clusters.loc[(year, slice(None)), 'Cluster'] = kmeans.fit_predict(df_percent_year)

display(df_clusters)

Cluster  total deaths cluster  Neoplasms cluster  \
year country                                                            
1994 Austria               7                   3.0                7.0   
     Belgium               9                   2.0                9.0   
     Switzerland           4                   0.0                9.0   
     Czech Republic        5                   6.0                5.0   
     Germany               7                   3.0                7.0   
...                      ...                   ...                ...   
2020 Romania               1                   8.0                1.0   
     Serbia                5                   9.0                1.0   
     Sweden                0                   1.0                6.0   
     Slovenia              7                   4.0                8.0   
     Slovakia              2                   5.0                4.0   

                     Diseases of the circulatory system cluster  \
year country                                                      
1994 Austria                                                7.0   
     Belgium                                                0.0   
     Switzerland                                            2.0   
     Czech Republic                                         9.0   
     Germany                                                5.0   
...                                                         ...   
2020 Romania                                                8.0   
     Serbia                                                 7.0   
     Sweden                                                 2.0   
     Slovenia                                               4.0   
     Slovakia                                               6.0   

                     Diseases of the respiratory system cluster  \
year country                                                      
1994 Austria                                                1.0   
     Belgium                                                6.0   
     Switzerland                                            4.0   
     Czech Republic                                         1.0   
     Germany                                                3.0   
...                                                         ...   
2020 Romania                                                5.0   
     Serbia                                                 3.0   
     Sweden                                                 3.0   
     Slovenia                                               1.0   
     Slovakia                                               5.0   

                     Diseases of the nervous system and the sense organs cluster  \
year country                                                                       
1994 Austria                                                       4.0             
     Belgium                                                       9.0             
     Switzerland                                                   8.0             
     Czech Republic                                                1.0             
     Germany                                                       5.0             
...                                                                ...             
2020 Romania                                                       1.0             
     Serbia                                                        2.0             
     Sweden                                                        7.0             
     Slovenia                                                      4.0             
     Slovakia                                                      1.0             

                     Endocrine, nutritional and metabolic diseases cluster  \
year country                                                                 
1994 Austria                                                       5.0       


# Grouping the classes
The variable T controls how many columns need to be in the same class to consider two countries as belonging to the same group. So if we set T=5, five different causes need to be simillar between the two countries, to say the two countries are simillar. The code is pretty confusing, but I hope the final container group_per_year is straighforward to use. I provided examples how to access the elements. Unfortunately to code is a bit slow (10s)

In [46]:
T = 5
groups_per_year = {}
for year in df_clusters.index.get_level_values('year').unique():
    groups = {}
    group_counter = 0
    for country in df_clusters.loc[year].index.get_level_values('country').unique():
        added_to_group = False
        if (group_counter==0):
            groups[group_counter] = [[country]]
            group_counter+=1
            added_to_group = True
        else:
            for i in range(group_counter):
                similarity = df_clusters.loc[(year, country), :] == df_clusters.loc[(year, groups[i][0][0]), :]
                if (similarity.sum() >= T):
                    if (len(groups[i][0])==1):
                        groups[i].append(similarity)
                    able_to_add = True
                    for group_country in groups[i][0][1:]:
                        similarity_group = df_clusters.loc[(year, country), :] == df_clusters.loc[(year, group_country), :]
                        if (groups[i][1] & similarity_group).sum() < T:
                            able_to_add = False
                            break
                    if able_to_add:
                        groups[i][0].append(country)
                        added_to_group = True
        if (not added_to_group):
            groups[group_counter] = [[country]]
            group_counter+=1
    for i in range(group_counter):
        if (len(groups[i])==2):
            cols = groups[i][1]
            groups[i][1] = list(df_clusters[cols.index[cols]].columns)
    groups_per_year[year] = groups
    # print(year, groups)



In [47]:
# Usage of groups_per_year:
print(groups_per_year[1998]) # gives the countries grouped by their columns of the year 1998
print(groups_per_year[1998][0]) # gives the group 0 of countries grouped by their columns of the year 1998
print(groups_per_year[1998][0][0]) # gives the countries of group 0 of countries grouped by their columns of the year 1998
print(groups_per_year[1998][0][1]) # gives the simillar causes of group 0 of countries grouped by their columns of the year 1998

{0: [['Austria', 'Belgium'], ['Cluster', 'total deaths cluster', 'Endocrine, nutritional and metabolic diseases cluster', 'Diseases of the digestive system cluster', 'COVID-19 cluster', 'Intentional self-harm cluster']], 1: [['Bulgaria', 'Hungary'], ['Diseases of the respiratory system cluster', 'Infectious and parasitic diseases cluster', 'Diseases of the blood cluster', 'Diseases of the skin and subcutaneous tissue cluster', 'COVID-19 cluster', 'Assault cluster']], 2: [['Switzerland', 'Spain'], ['Neoplasms cluster', 'Endocrine, nutritional and metabolic diseases cluster', 'Mental and behavioural disorders cluster', 'COVID-19 cluster', 'Accidents cluster', 'Assault cluster']], 3: [['Czech Republic', 'Estonia'], ['Diseases of the circulatory system cluster', 'Diseases of the blood cluster', 'Mental and behavioural disorders cluster', 'Abuse of alcohol cluster', 'Abuse of drugs cluster', 'COVID-19 cluster']], 4: [['Germany', 'Denmark'], ['Diseases of the nervous system and the sense org